In [1]:
import numpy as np
import matplotlib.pyplot as plt
import optimap as om
import ca_cardiac_model                                                                   
import math
import random


In [2]:
# Set parameters from the provided defaults
lx = 100  # Assuming a small grid size, adjust as needed
ly = 100  # Assuming a small grid size, adjust as needed
nstim = 2  # Number of stimuli, adjust as needed
iseed = 823323  # Initial random number seed
rbcl = 1000.0  # Pacing rate
dfu = 0.0001  # Effective voltage diffusion coefficient

# Ionic current parameters
gicai = 2.20  # Strength of LCC
gtos = 0.04  # Strength of ito slow
gtof = 0.15  # Strength of ito fast
gnacai = 1.5  # Strength of NCX

zxr = 0.09  # Controls degree of Ca-induced inactivation

nbt = 4000  # Total number of RyR2 clusters
cxinit = 1200.0  # Initial SR load

# Sodium concentration calculation
xmx = -2.0/250.0
xnai = xmx * 520.0 + 16.0  # Constant Na concentration

# Constants
xnao = 136.0  # External Na (mM)
xki = 140.0  # Internal K (mM)
xko = 5.40  # External K (mM)
cao = 1.8  # External Ca (mM)

temp = 308.0  # Temperature (K)
xxr = 8.314  # Gas constant
xf = 96.485  # Faraday's constant

dt = .5  # Time step
parallel = False # RK4

mod_output = 10 

nstep = rbcl/dt 

max_buffer_size = math.ceil((nstep*nstim) /mod_output) + (nstim)


def stimulus_function(ix, iy, time):
    # Example: Stimulate a 10x10 corner for the first 1ms
    if time < 1.0 and ix < 5 and iy < 5:
        return 80.0
    else:
        return 0.0


In [3]:
# Global variables to maintain state between calls
last_time_checked = -1
stimulus_centers = []
stimulus_active = False
stimulus_start_time = 0

def localized_random_stimulus(ix, iy, time, radius=5, stim_duration=1.0, time_prob=0.01):
    global last_time_checked, stimulus_centers, stimulus_active, stimulus_start_time
    
    # Check if we're at a new time point
    if abs(time - last_time_checked) > 1e-6:  # Floating point comparison
        last_time_checked = time
        
        # If stimulus is active, check if it should end
        if stimulus_active and (time - stimulus_start_time > stim_duration):
            stimulus_active = False
        
        # Randomly decide to start a new stimulus
        if not stimulus_active and random.random() < time_prob:
            stimulus_active = True
            stimulus_start_time = time
            # Choose a random center point
            center_x = random.randint(1, lx)
            center_y = random.randint(1, ly)
            stimulus_centers = [(center_x, center_y)]
    
    # Check if current point is within the stimulus radius of any center
    
    if stimulus_active:
        #print(stimulus_active)
        for center_x, center_y in stimulus_centers:
            distance = ((ix - center_x)**2 + (iy - center_y)**2)**0.5
            if distance <= radius:
                return 80.0  # Return stimulus current
    
    return 0.0  # No stimulus

In [1]:
import ca_cardiac_model
print(ca_cardiac_model)

v_out, cb_out, csrb_out, ci_out, t_out, num_steps = ca_cardiac_model.cardiac_simulation(
        lx, ly, nstim, iseed, rbcl, dfu, gicai, gtos, gtof, gnacai, zxr, nbt, 
        cxinit, xnai, xnao, xki, xko, cao, temp, xxr, xf, dt,
        max_buffer_size,mod_output,parallel,localized_random_stimulus
    )


: 

: 